## 4.1.6 재현 신경망(Recurrent Neural Network) 분류 모델

In [33]:
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import os
import json

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## 학습 데이터 파일 로드

In [34]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [35]:
# 넘파이 형태로 넘어온 파일을 오픈하여 학습 데이터를 구성한다.
input_data = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
label_data = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = None

# 단어 사전은 json으로 저장했으므로 json으로 불러온다.
with open(DATA_IN_PATH + DATA_CONFIGS, 'r') as f:
    prepro_configs = json.load(f)

In [36]:
TEST_SPLIT = 0.1
RANDOM_SEED = 13371447

# 학습 데이터와 평가 데이터로 나눈다.
train_input, test_input, train_label, test_label = train_test_split(input_data, label_data, 
                                                                    test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [37]:
BATCH_SIZE = 16
NUM_EPOCHS = 3

# 데이터 입력 함수 구현

# 매핑 함수
def mapping_fn(X, Y=None):
    inputs, labels = {'x': X}, Y
    return inputs, labels

# 학습 데이터 입력 함수
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    dataset = dataset.shuffle(buffer_size=50000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

# 평가 데이터 입력 함수
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input, test_label))
    dataset = dataset.map(mapping_fn)
    dataset = dataset.batch(BATCH_SIZE * 2)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

## 모델 정의

In [38]:
# 모델 하이퍼파라미터 정의

# 단어 사전에 대한 크기
VOCAB_SIZE = prepro_configs['vocab_size']+1
# 각 변수의 차원과 학습률
WORD_EMBEDDING_DIM = 100
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150

learning_rate = 0.001

In [39]:
print(len(prepro_configs['vocab']), VOCAB_SIZE)


74065 74066


In [40]:
# 모델 함수
def model_fn(features, labels, mode):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    # 임베딩 층
    
    # 입력값을 임베딩 벡터 형태로 만든다.
    embedding_layer = tf.keras.layers.Embedding(
                    VOCAB_SIZE,
                    WORD_EMBEDDING_DIM)(features['x'])
    
    # 임베딩 벡터에 20%의 dropout 적용
    embedding_layer = tf.keras.layers.Dropout(0.2)(embedding_layer)
    
    # 순환 신경망 구현
    
    # 심층 순환 신경망 모델은 LSTM모델을 통해 구현
    
    # 순환 신경망 구현을 위해 RNNCell이란 객체를 활용한다.
    # RNNCell은 순환 신경망 객체라고 보면 된다.
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
    # 여러 LSTMCell을 쌓게되면 하나의 MultiRNN으로 래핑해야한다.
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    # 네트워크와 임베딩 벡터와 연산하기 위해 dynamic_rnn함수를 선언한다.
    # RNNCell객체는 시퀀스 한 스텝에 대한 연산만 가능하다. 따라서 여러 스텝을 연산하기 위해 for문을 활용해야한다.
    # dynamic_rnn함수는 for문 없이 자동으로 순환 신경만을 만들어 주는 역할을 한다.
    # 입력 인자로는 cell에 순환 신경망 객체, inputs에는 입력값, dtype에는 출력값의 데이터 타입을 넣는다.
    outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                       inputs=embedding_layer,
                                       dtype=tf.float32)
    
    # 출력값에도 20%의 dropout을 적용한다.
    outputs = tf.keras.layers.Dropout(0.2)(outputs)
    # Dense에 적용시키는 입력값은 LSTM 신경망의 마지막 출력값을 넣어준다.
    hidden_layer = tf.keras.layers.Dense(DENSE_FEATURE_DIM, activation=tf.nn.tanh)(outputs[:,-1,:])
    # 이 값 또한 20%의 droptout을 적용한다.
    hidden_layer = tf.keras.layers.Dropout(0.2)(hidden_layer)
    
    # 최종적으로 1dim의 output만 할 수 있도록 dense layer를 활용해 차원 변환을 한다.
    logits = tf.keras.layers.Dense(1)(hidden_layer)
    logits = tf.squeeze(logits, axis=-1)
    
    # 시그모이드 함수를 적용해 0과 1사이의 값으로 바꾼다.
    sigmoid_logits = tf.nn.sigmoid(logits)
    
    # 모델 예측을 위한 구현
    # 모델 성능을 어느 정도 평가하고 나면 캐글에 평가할 데이터를 가지고 예측해야 한다.
    if PREDICT:
        # 시그모이드 함수가 적용된 모델 출력값을 dict 객체로 표현해서 에스티메이터에서 반환할 예측값을 만든다.
        predictions = {'sentiment': sigmoid_logits}
        
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  predictions=predictions)
    
    # 모델에서 구현한 값과 정답 라벨을 가지고 손실 값을 구한다.
    loss = tf.losses.sigmoid_cross_entropy(labels, logits)
    
    # 모델 평가를 위한 구현
    if EVAL:
        # 감정 예측값이 정답 라벨과 얼마나 일치하는지 정확도를 본다.
        # 정확도는 tf.metrics.accuracy를 통해 나타낼 수 있다.
        accuracy = tf.metrics.accuracy(labels, tf.round(sigmoid_logits))
        eval_metric_ops = {'acc': accuracy}
        
        # accuracy값을 tf.estimator.EstimatorSpec의 eval_metric_ops인자로 입력하면 평가 결괏값을 확인할 수 있다.
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    # 모델 학습을 위한 구현
    if TRAIN:
        # 현재 학습 글로벌 스텝 얻기
        global_step = tf.train.get_global_step()
        # AdamOptimizer 객체 생성
        # 예측 손실값을 구하고 나면 파라미터 최적화를 하고자 경사도 하강법을 진행한다.
        # 여기서는 경사도 하강법으로 아담 옵티마이저를 활용한다.
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
        
        # 직접 모델 함수를 구현하게 되면 tf.estimator.EstimatorSpec 객체를 생성해서 반환하게 한다.
        # 이 객체는 현재 함수가 어느 모드에서 실행되고 있는지 확인한다. 그리고 모드에 따라 입력 인자가 다르다.
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [41]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# tf/estimator.Estimator객체 생성
# 인자는 작성한 모델함수를 넣는다.
est = tf.estimator.Estimator(model_fn=model_fn,
                             model_dir=DATA_OUT_PATH + 'checkpoint/rnn') 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# 모델 학습
est.train(train_input_fn)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:t

In [43]:
# 모델 평가
est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-24T20:22:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/checkpoint/rnn\model.ckpt-4221
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 3.57425s
INFO:tensorflow:Finished evaluation at 2020-07-24-20:22:09
INFO:tensorflow:Saving dict for global step 4221: acc = 0.5452, global_step = 4221, loss = 0.67958033
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4221: ./data_out/checkpoint/rnn\model.ckpt-4221


{'acc': 0.5452, 'loss': 0.67958033, 'global_step': 4221}

## 캐글 평가 데이터셋 만들기

In [44]:
DATA_OUT_PATH = './data_out/'
TEST_INPUT_DATA = 'test_input.npy'
TEST_ID_DATA = 'test_id.npy'

test_input_data = np.load(open(DATA_IN_PATH + TEST_INPUT_DATA, 'rb'))

In [45]:
# tf.estimator.inputs.numpy_input_fn함수를 활용해 데이터 입력 함수 생성
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_input_data}, shuffle=False)

In [46]:
# estimator.predict로 모델을 통해 데이터를 예측한다.
predictions = np.array([p['sentiment'] for p in est.predict(input_fn=
predict_input_fn)])

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/checkpoint/rnn\model.ckpt-4221
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [47]:
# 제출을 위해 평가 데이터의 id값을 불러온다.
test_id = np.load(open(DATA_IN_PATH + TEST_ID_DATA, 'rb'))

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

output = pd.DataFrame(data={"id": list(test_id), "sentiment":list(predictions)} )
output.to_csv(DATA_OUT_PATH + 'movie_review_result_rnn.csv', index=False, quoting=3 )